# Import Library

In [ ]:
import pandas as pd
import sqlite3
import requests
from bs4 import BeautifulSoup
import seaborn as sns
import matplotlib.pyplot as plt

# Extract Data

In [ ]:
# From csv
electricity_access_precent = pd.read_csv('DataSource/electricity_access_precent.csv', quotechar='"')

rural_population_percent = pd.read_csv("DataSource/rural_population_percent.csv", quotechar='"')

gdp_data = pd.read_csv('DataSource/gdp_data.csv', quotechar='"')

mystery = pd.read_csv('DataSource/mystery.csv', quotechar='"')

population_data_from_csv = pd.read_csv('DataSource/population_data.csv', quotechar='"')

projects_data = pd.read_csv("DataSource/projects_data.csv", quotechar='"')


# From json
population_data_from_json = pd.read_json('DataSource/population_data.json')


# From db
conn = sqlite3.connect('DataSource/population_data.db')
population_data_from_db = pd.read_sql_query("SELECT * FROM population_data;", conn)
conn.close()


# From xml
with open("DataSource/population_data.xml", "r") as f:
    xml_data = f.read()

soup = BeautifulSoup(xml_data, "xml")
data = []
for record in soup.find_all("record"):
    record_data = {}
    for field in record.find_all("field"):
        name = field.get("name")
        value = field.text
        key = field.get("key")  
        record_data[name] = value
        if name != "Year" and name != "Value": 
            record_data[name+"_key"] = key  
    data.append(record_data)

population_data_from_xml = pd.DataFrame(data)


# from API
all_iso2codes = []

url = "https://api.worldbank.org/v2/countries/"
response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.content, "xml")
    total_pages = int(soup.find('wb:countries')['pages'])
    for page_num in range(1, total_pages + 1):
        url = f"https://api.worldbank.org/v2/countries/?page={page_num}"
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "xml")
            countries = soup.find_all('wb:country')
            iso2codes = [country.find('wb:iso2Code').text for country in countries]
            all_iso2codes.extend(iso2codes)
        else:
            print("Error:", response.status_code)
else:
    print("Error:", response.status_code)

iso2codes_combined = ";".join(all_iso2codes)

url = f"https://api.worldbank.org/v2/countries/{iso2codes_combined}/indicators/SP.POP.TOTL/?format=json&per_page=1000"
data = []
page = 1
total_pages = None

while total_pages is None or page <= total_pages:
    response = requests.get(url, params={"page": page})
    response_data = response.json()
    if total_pages is None:
        total_pages = response_data[0]['pages']
    data.extend(response_data[1])
    page += 1

df_worldbank = pd.json_normalize(data)

# Transfrom

#### - Check Duplicate

In [ ]:
duplicate_rows = electricity_access_precent[projects_data.duplicated(keep=False)]
print(duplicate_rows)

duplicate_rows = gdp_data[projects_data.duplicated(keep=False)]
print(duplicate_rows)

duplicate_rows = mystery[projects_data.duplicated(keep=False)]
print(duplicate_rows)

duplicate_rows = population_data_from_csv[projects_data.duplicated(keep=False)]
print(duplicate_rows)

duplicate_rows = population_data_from_db[projects_data.duplicated(keep=False)]
print(duplicate_rows)

duplicate_rows = population_data_from_json[projects_data.duplicated(keep=False)]
print(duplicate_rows)

duplicate_rows = population_data_from_xml[projects_data.duplicated(keep=False)]
print(duplicate_rows)

duplicate_rows = projects_data[projects_data.duplicated(keep=False)]
print(duplicate_rows)

duplicate_rows = rural_population_percent[projects_data.duplicated(keep=False)]
print(duplicate_rows)

duplicate_rows = df_worldbank[projects_data.duplicated(keep=False)]
print(duplicate_rows)

#### - Handle Missing Value

In [ ]:
def handle_missing_value_by_column(data):
    total_row = len(data)
    missing_values = data.isnull().sum()
    missing_percentage = (missing_values / (missing_values + total_row)) * 100
    columns_to_drop = missing_percentage[missing_percentage > 20].index
    data.drop(columns=columns_to_drop, inplace=True)
    return data
                

def handle_missing_value_by_row(data, column_param) :
    index = data.columns.get_loc(column_param) ## ambil index dari kolom pembatas
    cek_kolom = data.columns[index + 1:]
    jumlah_cek_kolom = len(cek_kolom)
    drop_rows = []
    for index, row in data.iterrows() :
        missing_values = row[cek_kolom].isna() | (row[cek_kolom]==0)
        if missing_values.any() :
            missing_values_count = missing_values.sum()
            missing_percent = (missing_values_count/jumlah_cek_kolom)*100
            if missing_percent > 20 :
                drop_rows.append(index)
            else:
                row[cek_kolom] = row[cek_kolom].replace(0, pd.NA)
                numeric_row = row[cek_kolom].apply(pd.to_numeric, errors='coerce')
                mean_row = numeric_row.mean()
                data.loc[index, cek_kolom] = numeric_row.fillna(mean_row)

    data.drop(index=drop_rows, inplace=True)
    return data

##### 1. electricity_access_percent

In [ ]:
handle_missing_value_by_column(electricity_access_precent)
handle_missing_value_by_row(electricity_access_precent, 'Indicator Code')
df = electricity_access_precent
df = df.rename(columns={'Country Name':'country_name','Country Code':'country_code', 'Indicator Name':'indicator_name','Indicator Code':'indicator_code'})
df = df.melt(id_vars=["country_name","country_code","indicator_name","indicator_code"], var_name='year', value_name='electricity_access_percent')
df = df.sort_values(by=['country_name', 'year'])
electricity_access_precent=df

##### 2. rural_population_percent

In [ ]:
handle_missing_value_by_column(rural_population_percent)
handle_missing_value_by_row(rural_population_percent, 'Indicator Code')
df = rural_population_percent
df = df.rename(columns={'Country Name':'country_name','Country Code':'country_code', 'Indicator Name':'indicator_name','Indicator Code':'indicator_code'})
df = df.melt(id_vars=["country_name","country_code","indicator_name","indicator_code"], var_name='year', value_name='rural_population_percent')
df = df.sort_values(by=['country_name','year'])
rural_population_percent=df

##### 3. All Population Data

In [8]:
# csv
df = population_data_from_csv
handle_missing_value_by_column(df)
handle_missing_value_by_row(df,'Indicator Code')
df['Indicator Name'] = df['Indicator Name'].str.replace('Population, total', 'Population total')
df = df.rename(columns={"Country Name":"country_name","Country Code":"country_code","Indicator Name":"indicator_name","Indicator Code":"indicator_code"})
df = df.melt(id_vars=["country_name","country_code","indicator_name","indicator_code"], var_name='year', value_name='population')
df = df.sort_values(by=['country_name','year'])
population_data_from_csv = df

# json
df = population_data_from_json
handle_missing_value_by_column(df)
handle_missing_value_by_row(df,'Indicator Code')
df['Indicator Name'] = df['Indicator Name'].str.replace('Population, total', 'Population total')
df = df.rename(columns={"Country Name":"country_name","Country Code":"country_code","Indicator Name":"indicator_name","Indicator Code":"indicator_code"})
df = df.melt(id_vars=["country_name","country_code","indicator_name","indicator_code"], var_name='year', value_name='population')
df = df.sort_values(by=['country_name', 'year'])
population_data_from_json = df

# db
df = population_data_from_db
df = df.drop(columns=['index'])
handle_missing_value_by_column(df)
handle_missing_value_by_row(df,'Indicator_Code')
df['Indicator_Name'] = df['Indicator_Name'].str.replace('Population, total', 'Population total')
df = df.rename(columns={"Country_Name":"country_name","Country_Code":"country_code","Indicator_Name":"indicator_name","Indicator_Code":"indicator_code"})
df = df.melt(id_vars=["country_name","country_code","indicator_name","indicator_code"], var_name='year', value_name='population')
df = df.sort_values(by=['country_name','year'])
population_data_from_db = df

# xml
df = population_data_from_xml
df = df[df['Country or Area'] != ['Not classified','Eritrea','Kuwait','West Bank and Gaza','Serbia','Sint Maarten (Dutch part)']]
df['Item'] = df['Item'].str.replace('Population, total', 'Population total')
df = df.rename(columns={"Country or Area":"country_name","Country or Area_key":"country_code","Item":"indicator_name","Item_key":"indicator_code", "Year":"year","Value": "population"})
population_data_from_xml = df


KeyError: 'Indicator Code'

##### 4. gdp_data

In [ ]:
handle_missing_value_by_column(gdp_data)
handle_missing_value_by_row(gdp_data, 'Indicator Code')
df = gdp_data
df = df.rename(columns={"Country Name":"country_name","Country Code":"country_code","Indicator Name":"indicator_name","Indicator Code":"indicator_code"})
df = df.melt(id_vars=["country_name","country_code","indicator_name","indicator_code"], var_name='year', value_name='current_usd')
df = df.sort_values(by=['country_name','year'])
gdp_data = df
# gdp_data


##### 5. mystery

In [ ]:
df = mystery
df = df.drop(columns=['Unnamed: 0'])
handle_missing_value_by_column(df)
handle_missing_value_by_row(df,'Indicator Code')
df['Indicator Name'] = df['Indicator Name'].str.replace('Population, total', 'Population total')
df = df.rename(columns={"Country Name":"country_name","Country Code":"country_code","Indicator Name":"indicator_name","Indicator Code":"indicator_code"})
df = df.melt(id_vars=["country_name","country_code","indicator_name","indicator_code"], var_name='year', value_name='population')
df = df.sort_values(by=['country_name', 'year'])
mystery = df

##### 6. projects_data

In [ ]:
## soon

#### - Combine Data

##### 1. merge rural_population_percent + electricity_access_percent

In [ ]:
combine_electricity_rural = pd.merge(rural_population_percent,electricity_access_precent, how='inner', on=["country_name", "country_code", "year"], suffixes=('_rural', '_electricity'))
combine_electricity_rural

In [ ]:
print(combine_electricity_rural.head())

##### 2. concat all data population

In [ ]:
concat_all_population = pd.concat([population_data_from_csv, population_data_from_db, population_data_from_json, mystery])
df = concat_all_population
df.set_index('country_name', inplace=True)
df.sort_values(by=['country_name', 'year'])
concat = df.drop_duplicates()
concat.to_csv('berhasil.csv')

##### 3. merge all population + gdp

In [ ]:
combine_pop_gdp = pd.merge(concat_all_population, gdp_data, how='inner', on=["country_name", "country_code", "year"],suffixes=['_population','_gdp'])
combine_pop_gdp

#### - Handle Outliers

In [ ]:
def handle_outliers(data, column):
    data = data.sort_values(by=column)
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1

    # Tentukan Batas Bawah dan Batas Atas
    lower_bound = Q1 - (1.5 * IQR)
    upper_bound = Q3 + (1.5 * IQR)

    # Identifikasi outliers
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]

    # Hapus outliers dari DataFrame
    data = data.drop(outliers.index)

    # print("DataFrame without outliers:")
    # print(df)
    return data

##### 1. All data population

In [ ]:
# csv
# json
# db
# xml

##### 2. 

In [ ]:
# sns.boxplot(electricity_access_precent['electricity_access_percent'])